In [ ]:
#############################################################
# Compile `main.cu` to `file.exe`
# Commented out because we don't want to run it all the time
#############################################################
!nvcc -w functions.cu main.cu -o file

In [ ]:
#############################################################
# Load python dependencies
# If package cannot be resolved from pylance, reload the interpreter
#############################################################
import numpy as np
import matplotlib.pyplot as plt
import utils

# Generate detuning
todo: distribution functions
read into detuning.dat

In [ ]:
#############################################################
# Generate and write detuning data to file
#############################################################
# Write 1D numpy array to file
detuning_values_1d = utils.gen_same_pop(2, 1000, None)
print("Detuning data:", detuning_values_1d)
np.savetxt("Detuning.dat", detuning_values_1d, delimiter="\t")

# Example: write 2D numpy array to file
# values_2d = np.array([
#     [10.0, 20.0, 40.0],
#     [10.0, 20.0, 50.0],
#     [10.0, 20.0, 60.0]
# ])
# np.savetxt("values_2d.dat", values_2d, delimiter="\t")

# Run simulation

To run a single system, use terminal: 

         k  n_tot theta,phi g gamma chi kap t_end t_num handle

file.exe 20 100000 0.5 0.0 1.6 0.0 0.0 160 0.5 600000 k20N5_0.5_0_kappa160

## Suggested file naming convention:
k{num_ens}N{order of magnitude of n_total}_{theta_phi}_{other params}

E.g., k5N5_1_0_kappa160

The below block of code is useful for running multiple jobs

In [ ]:
num_ens_range = [2]
max_det_range = [0,100,1000,5000]
det_range_manual = []
sym_range = [False]
N_total_range = [10000]
theta_0_range = [1]
phi_0_range = [0.0]
coup_a_0_range = [1.0]
gamma_a_0_range = [0.0]
chi_a_0_range = [0.0]
kappa_c_0_range = [100.0]
t_max_range = [0.2]
t_num_range = [60000]
det_dict = {}
handle_list = []

runs = 0
for num_ens in num_ens_range:
    for max_det in max_det_range:
        for sym in sym_range:
            detuning_values_1d = utils.gen_same_pop(num_ens, max_det, sym)
            FWHM = detuning_values_1d[int(num_ens/2)]
            np.savetxt(f"Detuning.dat", detuning_values_1d, delimiter="\t")
            for N_total in N_total_range:
                for theta_0 in theta_0_range:
                    for phi_0 in phi_0_range:
                        for coup_a_0 in coup_a_0_range:
                            for gamma_a_0 in gamma_a_0_range:
                                for chi_a_0 in chi_a_0_range:
                                    for kappa_c_0 in kappa_c_0_range:
                                        for t_max in t_max_range:
                                            for t_num in t_num_range:
                                                runs += 1
                                                params = f'{num_ens} {N_total} {theta_0} {phi_0} {coup_a_0} {gamma_a_0} {chi_a_0} {kappa_c_0} {t_max} {t_num}'
                                                handle = f'k{num_ens}N{N_total}det{max_det}_{theta_0}_{phi_0}_{sym}_{runs}' # Add extra notes as needed
                                                det_dict[handle] = detuning_values_1d
                                                handle_list.append(handle)
                                                print(f"************************* Run {runs} *************************")
                                                print(f"FWHM: {FWHM}, handle: {handle} ")
                                                utils.runcmd(f'file.exe {params} {handle}')
                                                print("\n")

In [ ]:
#############################################################
# Test 2: effect of inhomogeneous broadening (intermediate case)
#############################################################
max_det = 280
detunings = [[0],[0,max_det/2],[0,max_det/2,max_det]]
num_ens = 3
N_total = 6000
theta_0 = 1
phi_0 = 0.0
coup_a_0 = 1.0
gamma_a_0 = 0.0
chi_a_0 = 0.0
kappa_c_0 = 100.0
t_max = 0.2
t_num = 60000
run = 0
det_dict = {}
handle_list = []

for i in range(len(detunings)):
    np.savetxt(f"Detuning.dat", detunings[i], delimiter="\t")
    params = f'{i+1} {2000*(i+1)} {theta_0} {phi_0} {coup_a_0} {gamma_a_0} {chi_a_0} {kappa_c_0} {t_max} {t_num}'
    handle = f'k{num_ens}N{N_total}det{max_det}_{theta_0}_{coup_a_0}_{kappa_c_0}_{i+1}' # Add extra notes as needed
    det_dict[handle] = detunings[i]
    handle_list.append(handle)
    print(f"************************* Run {i+1} *************************")
    print(f"Handle: {handle} ")
    utils.runcmd(f'file.exe {params} {handle}')
    print("\n")

# Analyze & Plot
todo:  import util functions return predict Td
write block to plot with Td and found Td

In [ ]:
#############################################################
# Read in the results file
# Returns: 
#         [0]: time steps
#         [1]: sz
#         [2]: coherence
#         [3]: photons
#############################################################
results = []
detunings = []
for i in range(np.size(handle_list)):
    results.append(utils.read_results(handle_list[i])) # Input: file handle
    detunings.append(det_dict[handle_list[i]])
# results = utils.read_results(handle_list[0]) # Input: file handle
print(detunings)

In [ ]:
#############################################################
# Plots intracavity field
#############################################################
for i in range(np.size(handle_list)):
    end_ind = utils.cut_time(results[0][0], -1)
    print(end_ind)
    plt.plot(results[i][0], results[i][3])
    plt.title("Intracavity photon", fontsize=12)
    plt.xlabel(r"Time ($\mu$s)", fontsize=12)
    plt.ylabel(r"$\langle a^\dagger a\rangle$", fontsize=12)

In [ ]:
#############################################################
# Line plots for inversion and coherence
#############################################################
plt.plot(results[0][0], results[0][1], label=detunings[0])
# Td = utils.findTd(results[0][1], results[0][0])
# print(Td)
plt.title("Atomic inversion", fontsize=12)
plt.xlabel(r"Time ($\mu$s)", fontsize=12)
plt.ylabel(r"$\langle \sigma_Z \rangle$", fontsize=12)
plt.legend()

In [ ]:
plt.plot(results[0][0], results[0][2], label=detunings[0])
plt.title("Coherence", fontsize=12)
plt.xlabel(r"Time ($\mu$s)", fontsize=12)
plt.ylabel(r"$\langle \sigma_+\sigma_- \rangle$", fontsize=12)
plt.legend()

In [ ]:
utils.plot_heat(results[0],detuning_values_1d[:20], np.rot90(results[1]), -1.0,1.0, 
                "Atom inversion $\langle\sigma_Z\\rangle$", "Time ($\mu$s)", "Detuning (kHz)")

In [ ]:
utils.plot_heat(results[0],detuning_values_1d, np.rot90(results[2]), 0,0.25, 
                "Coherences", "Time ($\mu$s)", "$\langle\sigma_+\sigma_-\\rangle$")

In [ ]:

a = np.rot90(np.asarray([[0,1,2],[3,4,5]]))
print(a)
utils.plot_heat([0,1], [3,2,1], a, 0,6, "na", "na", "na")